<a href="https://colab.research.google.com/github/manojach87/miscellaneous/blob/master/PySpark_MarvelRelationships.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 82kB/s 
     |████████████████████████████████| 204kB 36.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=e0fd3258fcd637c402d5f57b22d56a1758950b91144f6a5436aaef5f9d03f475
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [2]:
!wget http://bioinfo.uib.es/~joemiro/marvel/vert1.txt

--2021-04-02 03:25:59--  http://bioinfo.uib.es/~joemiro/marvel/vert1.txt
Resolving bioinfo.uib.es (bioinfo.uib.es)... 130.206.30.39
Connecting to bioinfo.uib.es (bioinfo.uib.es)|130.206.30.39|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182824 (179K) [text/plain]
Saving to: ‘vert1.txt’

vert1.txt           100%[===================>] 178.54K  --.-KB/s    in 0.1s    

2021-04-02 03:25:59 (1.40 MB/s) - ‘vert1.txt’ saved [182824/182824]



In [3]:
!wget http://bioinfo.uib.es/~joemiro/marvel/vert2.txt

--2021-04-02 03:25:59--  http://bioinfo.uib.es/~joemiro/marvel/vert2.txt
Resolving bioinfo.uib.es (bioinfo.uib.es)... 130.206.30.39
Connecting to bioinfo.uib.es (bioinfo.uib.es)|130.206.30.39|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 276188 (270K) [text/plain]
Saving to: ‘vert2.txt’

vert2.txt           100%[===================>] 269.71K  1.76MB/s    in 0.2s    

2021-04-02 03:26:00 (1.76 MB/s) - ‘vert2.txt’ saved [276188/276188]



In [4]:
!wget http://bioinfo.uib.es/~joemiro/marvel/porgat.txt

--2021-04-02 03:26:00--  http://bioinfo.uib.es/~joemiro/marvel/porgat.txt
Resolving bioinfo.uib.es (bioinfo.uib.es)... 130.206.30.39
Connecting to bioinfo.uib.es (bioinfo.uib.es)|130.206.30.39|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 924997 (903K) [text/plain]
Saving to: ‘porgat.txt’

porgat.txt          100%[===================>] 903.32K  3.34MB/s    in 0.3s    

2021-04-02 03:26:00 (3.34 MB/s) - ‘porgat.txt’ saved [924997/924997]



In [5]:
import pyspark as py

In [6]:
sc=py.SparkContext()

In [7]:
charactersPath="vert1.txt"
booksPath="vert2.txt"
edgesPath="porgat.txt"

characters=sc.textFile(charactersPath)
books=sc.textFile(booksPath)
edges=sc.textFile(edgesPath)

In [8]:
books.take(10)

['Vertex 6487: AA2 35',
 'Vertex 6488: M/PRM 35',
 'Vertex 6489: M/PRM 36',
 'Vertex 6490: M/PRM 37',
 'Vertex 6491: WI? 9',
 'Vertex 6492: AVF 4',
 'Vertex 6493: AVF 5',
 'Vertex 6494: H2 251',
 'Vertex 6495: H2 252',
 'Vertex 6496: COC 1']

In [9]:
characters.take(10)

['Vertex 1: 24-HOUR MAN/EMMANUEL',
 'Vertex 2: 3-D MAN/CHARLES CHANDLER & HAROLD CHANDLER',
 'Vertex 3: 4-D MAN/MERCURIO',
 'Vertex 4: 8-BALL/',
 'Vertex 5: A',
 "Vertex 6: A'YIN",
 'Vertex 7: ABBOTT, JACK',
 'Vertex 8: ABCISSA',
 'Vertex 9: ABEL',
 'Vertex 10: ABOMINATION/EMIL BLONSKY']

In [10]:
edges.take(10)

['*Vertices 19428 6486',
 '1 "24-HOUR MAN/EMMANUEL"',
 '2 "3-D MAN/CHARLES CHAN"',
 '3 "4-D MAN/MERCURIO"',
 '4 "8-BALL/"',
 '5 "A"',
 '6 "A\'YIN"',
 '7 "ABBOTT, JACK"',
 '8 "ABCISSA"',
 '9 "ABEL"']

In [12]:
def edgeFilterRow(row):
  if( "*" in row or '"' in row):
    return False
  else:
    return True

edgesFiltered = edges.filter(edgeFilterRow)
edgesFiltered.take(10)

['1 6487',
 '2 6488 6489 6490 6491 6492 6493 6494 6495 6496',
 '3 6497 6498 6499 6500 6501 6502 6503 6504 6505',
 '4 6506 6507 6508',
 '5 6509 6510 6511',
 '6 6512 6513 6514 6515',
 '7 6516',
 '8 6517 6518',
 '9 6519 6520',
 '10 6521 6522 6523 6524 6525 6526 6527 6528 6529 6530 6531 6532 6533 6534 6535']

In [17]:
characterBookMap=edgesFiltered.map(lambda x:x.split(" "))\
  .map(lambda x:(x[0],x[1:]))
characterBookMap.take(10)

[('1', ['6487']),
 ('2',
  ['6488', '6489', '6490', '6491', '6492', '6493', '6494', '6495', '6496']),
 ('3',
  ['6497', '6498', '6499', '6500', '6501', '6502', '6503', '6504', '6505']),
 ('4', ['6506', '6507', '6508']),
 ('5', ['6509', '6510', '6511']),
 ('6', ['6512', '6513', '6514', '6515']),
 ('7', ['6516']),
 ('8', ['6517', '6518']),
 ('9', ['6519', '6520']),
 ('10',
  ['6521',
   '6522',
   '6523',
   '6524',
   '6525',
   '6526',
   '6527',
   '6528',
   '6529',
   '6530',
   '6531',
   '6532',
   '6533',
   '6534',
   '6535'])]

In [18]:
characters.take(10)

['Vertex 1: 24-HOUR MAN/EMMANUEL',
 'Vertex 2: 3-D MAN/CHARLES CHANDLER & HAROLD CHANDLER',
 'Vertex 3: 4-D MAN/MERCURIO',
 'Vertex 4: 8-BALL/',
 'Vertex 5: A',
 "Vertex 6: A'YIN",
 'Vertex 7: ABBOTT, JACK',
 'Vertex 8: ABCISSA',
 'Vertex 9: ABEL',
 'Vertex 10: ABOMINATION/EMIL BLONSKY']

In [24]:
def charParse(row):
  row=row.split(":")
  id=row[0][7:] #"Vertex 7777" gets converted to "7777"
  nm=row[1].strip() # trims the output
  return((id,nm))

charParse('Vertex 10: ABOMINATION/EMIL BLONSKY      ')

('10', 'ABOMINATION/EMIL BLONSKY')

In [29]:
characterLookup=characters.map(charParse).collectAsMap()
characterLookup

{'1': '24-HOUR MAN/EMMANUEL',
 '2': '3-D MAN/CHARLES CHANDLER & HAROLD CHANDLER',
 '3': '4-D MAN/MERCURIO',
 '4': '8-BALL/',
 '5': 'A',
 '6': "A'YIN",
 '7': 'ABBOTT, JACK',
 '8': 'ABCISSA',
 '9': 'ABEL',
 '10': 'ABOMINATION/EMIL BLONSKY',
 '11': 'ABOMINATION | MUTANT X-VERSE',
 '12': 'ABOMINATRIX',
 '13': 'ABRAXAS',
 '14': 'ADAM 3,031',
 '15': 'ABSALOM',
 '16': 'ABSORBING MAN/CARL CRUSHER CREEL',
 '17': 'ABSORBING MAN | MUTANT X-VERSE',
 '18': 'ACBA',
 '19': 'ACHEBE, REVEREND DOCTOR MICHAEL IBN AL-HAJJ',
 '20': 'ACHILLES',
 '21': 'ACHILLES II/HELMUT',
 '22': 'ACROBAT/CARL ZANTE',
 '23': 'ADAM X',
 '24': 'ADAMS, CINDY',
 '25': 'ADAMS, CONGRESSMAN HENRY',
 '26': 'ADAMS, GEORGE',
 '27': 'ADAMS, MARTHA',
 '28': 'ADAMS, NICOLE NIKKI',
 '29': 'ADAMSON, JASON',
 '30': 'ADAMSON, REBECCA',
 '31': 'ADMIRAL PROTOCOL/',
 '32': 'ADORA',
 '33': 'ADORA CLONE',
 '34': 'ADRIA',
 '35': 'ADVA',
 '36': 'ADVENT/KYLE GROBE',
 '37': 'ADVERSARY',
 '38': 'AEGIS/TREY ROLLINS',
 '39': 'AENTAROS',
 '40': 'AFTERLI

In [31]:
characterBookMap.mapValues(lambda x:len(x)).take(10)

[('1', 1),
 ('2', 9),
 ('3', 9),
 ('4', 3),
 ('5', 3),
 ('6', 4),
 ('7', 1),
 ('8', 2),
 ('9', 2),
 ('10', 15)]

In [33]:
characterBookMap.map(lambda x:(characterLookup[x[0]],x[0])).take(10)

[('24-HOUR MAN/EMMANUEL', '1'),
 ('3-D MAN/CHARLES CHANDLER & HAROLD CHANDLER', '2'),
 ('4-D MAN/MERCURIO', '3'),
 ('8-BALL/', '4'),
 ('A', '5'),
 ("A'YIN", '6'),
 ('ABBOTT, JACK', '7'),
 ('ABCISSA', '8'),
 ('ABEL', '9'),
 ('ABOMINATION/EMIL BLONSKY', '10')]

In [40]:
charaterStrength=characterBookMap.mapValues(lambda x:len(x))\
  .map(lambda x:(characterLookup[x[0]],x[1]))\
  .reduceByKey(lambda x,y:x+y)\
  .sortBy(lambda x:-x[1])
charaterStrength.take(10)

[('SPIDER-MAN/PETER PARKER', 1625),
 ('CAPTAIN AMERICA', 1367),
 ('IRON MAN/TONY STARK', 1168),
 ('THING/BENJAMIN J. GRIMM', 990),
 ('THOR/DR. DONALD BLAKE/SIGURD JARLSON II/JAKE OLSON/LOREN OLSON', 965),
 ('HUMAN TORCH/JOHNNY STORM', 908),
 ('MR. FANTASTIC/REED RICHARDS', 875),
 ('HULK/DR. ROBERT BRUCE BANNER', 841),
 ('WOLVERINE/LOGAN', 820),
 ('INVISIBLE WOMAN/SUE STORM RICHARDS', 782)]

In [47]:
bookCharacterMap =  characterBookMap.flatMapValues(lambda x : x)\
                                    .map(lambda x:(x[1],x[0]))\
                                    .reduceByKey(lambda x,y: x +","+y)\
                                    .mapValues(lambda x:x.split(","))
bookCharacterMap.take(2)

[('6490', ['2']),
 ('6493',
  ['2',
   '154',
   '859',
   '2186',
   '2399',
   '2555',
   '3035',
   '3491',
   '4228',
   '6022',
   '6148'])]

In [50]:
import itertools

cooccurenceMap=bookCharacterMap.flatMap(lambda x:list(itertools.combinations(x[1],2)))
cooccurenceMap.take(10)

[('2', '154'),
 ('2', '859'),
 ('2', '2186'),
 ('2', '2399'),
 ('2', '2555'),
 ('2', '3035'),
 ('2', '3491'),
 ('2', '4228'),
 ('2', '6022'),
 ('2', '6148')]

In [56]:
cooccurenceStrength=cooccurenceMap.map(lambda x:(x,1))\
                                .reduceByKey(lambda x,y: x+y)\
                                .sortBy(lambda x:-x[1])
cooccurenceStrength.take(10)

[(('2557', '5716'), 744),
 (('2557', '3805'), 713),
 (('3805', '5716'), 708),
 (('2650', '3805'), 701),
 (('2557', '2650'), 694),
 (('2650', '5716'), 668),
 (('5306', '6166'), 616),
 (('2959', '5306'), 526),
 (('859', '2664'), 446),
 (('4898', '6066'), 422)]

In [57]:
cooccurenceEdges = occurenceStrength.map(lambda x:(x[0][0],x[0][1],x[1]))
cooccurenceEdges.take(10)

[('2557', '5716', 744),
 ('2557', '3805', 713),
 ('3805', '5716', 708),
 ('2650', '3805', 701),
 ('2557', '2650', 694),
 ('2650', '5716', 668),
 ('5306', '6166', 616),
 ('2959', '5306', 526),
 ('859', '2664', 446),
 ('4898', '6066', 422)]